In [75]:
# import dependencies
import numpy as np
import scipy as sp
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib as jl
import datetime as dt
import requests

import os
from dotenv import load_dotenv

from ta import add_all_ta_features
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.volume import VolumeWeightedAveragePrice
from ta.momentum import StochRSIIndicator

1. go to that github code and just switch it for all my etfs
2. change to paper trading api

In [76]:
load_dotenv()
TOKEN = os.getenv('TOKEN')

In [77]:
# # create watchlist
# response = requests.post('https://api.tradier.com/v1/watchlists',
#     data={'name': 'CNBC_INDEX_LIST', 'symbols': 'SPY,QQQ,IWM,DIA,VTI,MDY,DBC,FEZ,OEF,IWF,IWD,PFF,VOO,IJH,IWO,IWN,ACWI,IEMG'},
#     headers={'Authorization': TOKEN, 'Accept': 'application/json'}
# )
# json_response = response.json()

In [78]:
# pull watchlist

response = requests.get('https://api.tradier.com/v1/watchlists/CNBC_INDEX_LIST',
    params={},
    headers={'Authorization': TOKEN, 'Accept': 'application/json'}
)
json_response = response.json()
print('Status Code:', response.status_code)

Status Code: 200


In [79]:
#tickers_df = pd.DataFrame.from_records(json_response['watchlist']['items']['item'])
#tickers = list(pd.read_csv('data/etf_list.csv').Symbol)

In [92]:
response = requests.get('https://api.tradier.com/beta/markets/fundamentals/financials',
    params={'symbols': 'AAPL'},
    headers={'Authorization': TOKEN, 'Accept': 'application/json'}
)
json_response = response.json()
print(response.status_code)
#print(json_response)

200


In [96]:
response = requests.get('https://eodhistoricaldata.com/api/fundamentals/VTI.US?api_token=63dc0e2f4efc43.34327983 '#,
    #params={'symbols': 'AAPL'},
    #headers={'Authorization': TOKEN, 'Accept': 'application/json'}
)
json_response = response.json()
print(response.status_code)
#print(json_response)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [95]:
print(json_response)

{'General': {'Code': 'VTI', 'Type': 'ETF', 'Name': 'Vanguard Total Stock Market Index Fund ETF Shares', 'Exchange': 'NYSE ARCA', 'CurrencyCode': 'USD', 'CurrencyName': 'US Dollar', 'CurrencySymbol': '$', 'CountryName': 'USA', 'CountryISO': 'US', 'Description': 'The fund employs an indexing investment approach designed to track the performance of the index, which represents approximately 100% of the investable U.S. stock market and includes large-, mid-, small-, and micro-cap stocks. It invests by sampling the index, meaning that it holds a broadly diversified collection of securities that, in the aggregate, approximates the full index in terms of key characteristics.', 'Category': 'Large Blend', 'UpdatedAt': '2023-02-02'}, 'Technicals': {'Beta': 1.0379, '52WeekHigh': 230.3389, '52WeekLow': 174.0015, '50DayMA': 197.4088, '200DayMA': 197.7753}, 'ETF_Data': {'ISIN': 'US9229087690', 'Company_Name': 'Vanguard', 'Company_URL': 'http://www.vanguard.com', 'ETF_URL': 'https://personal.vanguard.

In [89]:
dict1 = json_response[0]['results'][0]['tables']

df_test = pd.DataFrame(list(dict1['financial_statements_restate']['income_statement'][0]['period_12m'].items()))

In [84]:
tickers = ['SPY', 'QQQ']
d = {}

for i in range(len(tickers)): 

    response = requests.get('https://api.tradier.com/v1/markets/history',
        params={'symbol': tickers[i], 'interval': 'daily', 'start': '2019-01-01', 'end': '2023-01-01'},
        headers={'Authorization': TOKEN, 'Accept': 'application/json'}
    )
    json_response = response.json()
    #print("Response Status Code:", response.status_code)
    i = i + 1
    print("progress", i)
    for symbol in tickers:
        d[symbol] = pd.DataFrame.from_records(json_response['history']['day'], index='date')

        indicator_macd = MACD(close = d[symbol]['close'], window_slow=26, window_fast=12, window_sign=9, fillna=True)
        d[symbol]['macd'] = indicator_macd.macd()
        d[symbol]['macd_diff'] = indicator_macd.macd_diff()
        d[symbol]['macd_signal'] = indicator_macd.macd_signal()

        indicator_bb = BollingerBands(close = d[symbol]['close'], window=20, window_dev=2, fillna=True)
        d[symbol]['bb_mavg'] = indicator_bb.bollinger_mavg()
        d[symbol]['bb_hband'] = indicator_bb.bollinger_hband()
        d[symbol]['bb_lband'] = indicator_bb.bollinger_lband()
        d[symbol]['bb_hband_ind'] = indicator_bb.bollinger_hband_indicator()
        d[symbol]['bb_lband_ind'] = indicator_bb.bollinger_lband_indicator()

        indicator_vwap = VolumeWeightedAveragePrice(
            high = d[symbol]['high'],
            low = d[symbol]['low'],
            close = d[symbol]['close'],
            volume = d[symbol]['volume'],
            window=14, fillna=True)
        d[symbol]['vwap'] = indicator_vwap.volume_weighted_average_price()

        indicator_stochrsi = StochRSIIndicator(close = d[symbol]['close'], window=14, smooth1=3, smooth2=3, fillna=True)
        d[symbol]['stoch_rsi'] = indicator_stochrsi.stochrsi()
        d[symbol]['stochrsi_d'] = indicator_stochrsi.stochrsi_d()
        d[symbol]['stochrsi_k'] = indicator_stochrsi.stochrsi_k()

progress 1
progress 2


In [85]:
df = pd.concat(d.values(), axis=1, keys=d.keys())
dask_df = dd.from_pandas(df, npartitions=6)

In [87]:
df.tail()

SPY                                                         \
              open     high     low   close    volume      macd macd_diff   
date                                                                        
2022-12-23  265.47  268.040  263.53  267.36  39373584 -3.392306 -2.183973   
2022-12-27  266.74  266.850  263.03  263.58  38009005 -4.051257 -2.274339   
2022-12-28  263.17  265.470  259.73  260.10  47139043 -4.798968 -2.417640   
2022-12-29  262.96  267.405  262.25  266.44  45893567 -4.824337 -1.954407   
2022-12-30  263.62  266.410  262.29  266.28  37858277 -4.801999 -1.545655   

                                              ...         QQQ            \
           macd_signal   bb_mavg    bb_hband  ... macd_signal   bb_mavg   
date                                          ...                         
2022-12-23   -1.208334  280.8695  297.487281  ...   -1.208334  280.8695   
2022-12-27   -1.776918  279.9130  298.122859  ...   -1.776918  279.9130   
2022-12-28   -2.381328  278.8895  299.034808  ...   -2.381328  278.8895   
2022-12-29   -2.869930  277.5435  297.233738  ...   -2.869930  277.5435   
2022-12-30   -3.256344  276.1715  294.965473  ...   -3.256344  276.1715   

                                                                          \
              bb_hband    bb_lband bb_hband_ind bb_lband_ind        vwap   
date                                                                       
2022-12-23  297.487281  264.251719          0.0          0.0  278.426559   
2022-12-27  298.122859  261.703141          0.0          0.0  277.342623   
2022-12-28  299.034808  258.744192          0.0          0.0  276.126820   
2022-12-29  297.233738  257.853262          0.0          0.0  275.059930   
2022-12-30  294.965473  257.377527          0.0          0.0  273.997453   

                                            
           stoch_rsi stochrsi_d stochrsi_k  
date                                        
2022-12-23  0.041120   0.078599   0.087737  
2022-12-27  0.000000   0.058491   0.013707  
2022-12-28  0.000000   0.038383   0.013707  
2022-12-29  0.354071   0.048479   0.118024  
2022-12-30  0.348161   0.121936   0.234077  

[5 rows x 34 columns]